In [9]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
import optuna

import matplotlib.pyplot as plt

### Setting Up GPU as a training device

In [10]:
physical_devices = tf.config.list_physical_devices()

print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

### Loading Data

In [12]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()

import numpy as np
from sklearn.model_selection import train_test_split

# Concatenate train and test images
X = np.concatenate((X_train, X_test))
y = np.concatenate((Y_train, Y_test))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=20000, test_size=5000, random_state=1)

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], 10))

i = 0
for [val] in y_train:
  y_train_fixed[i][val] = 1
  i += 1
  
i = 0
for [val] in y_test:
  y_test_fixed[i][val] = 1
  i += 1

### Loading existing study

In [16]:
existing_study = joblib.load('optuna_study_saving_test.pkl')

existing_study.best_params

{'filters_1': 64,
 'kernel_size_1': 3,
 'filters_2': 64,
 'kernel_size_2': 5,
 'strides_2': 1,
 'max_pool_size_1': 3,
 'dense_size_1': 256,
 'learning_rate': 8.253374315877225e-05}

### Setting up the model

In [ ]:
import warnings
import joblib
from keras import layers

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 80

input_shape = (32, 32, 3)

In [ ]:
def objective(trial):
  model = Sequential(
    [
      keras.Input(shape=(32, 32, 3)),
      layers.Conv2D(
        filters=trial.suggest_categorical('filters_1', [32, 64, 96, 128]),
        kernel_size=trial.suggest_categorical('kernel_size_1', [3, 5]),
        padding='same', activation="relu"
      ),
      layers.Dropout(0.1),
      layers.Conv2D(
        filters=trial.suggest_categorical('filters_2', [32, 64, 96, 128]),
        kernel_size=trial.suggest_categorical('kernel_size_2', [3, 5]),
        strides=trial.suggest_categorical('strides_2', [1, 2]),
        padding='same', activation="relu"
      ),
      layers.Dropout(0.2),
      layers.MaxPooling2D(pool_size=trial.suggest_categorical('max_pool_size_1', [2, 3])),
      layers.Flatten(),
      layers.Dense(trial.suggest_categorical('dense_size_1', [64, 128, 256]), activation="relu"),
      layers.Dropout(0.4),
      layers.Dense(CLASSES, activation="softmax"),
    ]
  )

  # We compile our model with a sampled learning rate.
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  
  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  model.fit(
    X_train,
    y_train_fixed,
    validation_split=0.1,
    # validation_data=(x_valid, y_valid),
    shuffle=True,
    batch_size=BATCHSIZE,
    epochs=EPOCHS,
    verbose=False,
  )

  # Evaluate the model accuracy on the validation set.
  score = model.evaluate(X_test, y_test_fixed, verbose=0)
  return score[1]

### Hyperparameter tunning

In [ ]:
%%time
warnings.warn(
  "Recent Keras release (2.4.0) simply redirects all APIs "
  "in the standalone keras package to point to tf.keras. "
  "There is now only one Keras: tf.keras. "
  "There may be some breaking changes for some workflows by upgrading to keras 2.4.0. "
  "Test before upgrading. "
  "REF:https://github.com/keras-team/keras/releases/tag/2.4.0"
)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=3600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

### Saving the study to the file

In [ ]:
joblib.dump(study, "optuna_study_saving_test.pkl")